In [ ]:
import json
import datetime
import matplotlib.pyplot as plt
import ttkbootstrap as ttk
from ttkbootstrap.constants import *
from tkinter import messagebox, simpledialog, BOTH, CENTER

# Definir as classes Tipo, Lote e Validade
class Tipo:
    def __init__(self, nome):
        self.nome = nome

class Lote:
    def __init__(self, numero):
        self.numero = numero

class Validade:
    def __init__(self, data):
        self.data = datetime.datetime.strptime(data, "%d/%m/%Y").date()

# Definir a classe Produto
class Produto:
    def __init__(self, nome, quantidade, preco, tipo, lote, validade, codigo_barras):
        self.nome = nome
        self.quantidade = quantidade
        self.preco = preco
        self.tipo = tipo
        self.lote = lote
        self.validade = validade
        self.codigo_barras = codigo_barras

# Definir o estoque como uma lista vazia
estoque = []

# Função para cadastrar item no estoque
def cadastrar_item():
    nome = simpledialog.askstring("Cadastro de Item", "Digite o nome do item:")
    quantidade = simpledialog.askinteger("Cadastro de Item", "Digite a quantidade em estoque:")
    preco = simpledialog.askfloat("Cadastro de Item", "Digite o preço por unidade (em reais):") * 100  # Converter para centavos
    tipo_nome = simpledialog.askstring("Cadastro de Item", "Digite o tipo do produto:")
    tipo = Tipo(tipo_nome)
    lote_numero = simpledialog.askstring("Cadastro de Item", "Digite o número do lote:")
    lote = Lote(lote_numero)
    validade_data = simpledialog.askstring("Cadastro de Item", "Digite a data de validade (dd/mm/aaaa):")
    validade = Validade(validade_data)
    codigo_barras = simpledialog.askstring("Cadastro de Item", "Digite o código de barras:")

    # Verificar se o código de barras já existe
    for produto in estoque:
        if produto.codigo_barras == codigo_barras:
            messagebox.showerror("Erro", "Código de barras já cadastrado para outro produto.")
            return

    produto = Produto(nome, quantidade, preco, tipo, lote, validade, codigo_barras)
    estoque.append(produto)
    messagebox.showinfo("Sucesso", "Item cadastrado com sucesso.")
    salvar_estoque_json()  # Salvar após cadastrar um novo item

# Função para remover item do estoque
def remover_item():
    nome = simpledialog.askstring("Remover Item", "Digite o nome do item a ser removido:")
    for produto in estoque:
        if produto.nome == nome:
            estoque.remove(produto)
            messagebox.showinfo("Sucesso", "Item removido do estoque com sucesso.")
            salvar_estoque_json()  # Salvar após remover um item
            return
    messagebox.showerror("Erro", "Item não encontrado no estoque.")

# Função para adicionar item com código de barras
def adicionar_item_codigo_barras():
    codigo_barras = simpledialog.askstring("Adicionar Item", "Digite o código de barras:")
    quantidade = simpledialog.askinteger("Adicionar Item", "Digite a quantidade a ser adicionada:")
    for produto in estoque:
        if produto.codigo_barras == codigo_barras:
            produto.quantidade += quantidade
            messagebox.showinfo("Sucesso", "Quantidade atualizada com sucesso.")
            salvar_estoque_json()  # Salvar após adicionar um item pelo código de barras
            return
    messagebox.showerror("Erro", "Código de barras não encontrado.")

# Função para subtrair item com código de barras
def subtrair_item_codigo_barras():
    codigo_barras = simpledialog.askstring("Subtrair Item", "Digite o código de barras:")
    quantidade = simpledialog.askinteger("Subtrair Item", "Digite a quantidade a ser subtraída:")
    for produto in estoque:
        if produto.codigo_barras == codigo_barras:
            if produto.quantidade >= quantidade:
                produto.quantidade -= quantidade
                messagebox.showinfo("Sucesso", "Quantidade atualizada com sucesso.")
                salvar_estoque_json()  # Salvar após subtrair um item pelo código de barras
                return
            else:
                messagebox.showerror("Erro", "Não há quantidade suficiente para subtrair.")
                return
    messagebox.showerror("Erro", "Código de barras não encontrado.")

# Função para consultar item pelo código de barras
def consultar_item_codigo_barras():
    codigo_barras = simpledialog.askstring("Consultar Item", "Digite o código de barras:")
    for produto in estoque:
        if produto.codigo_barras == codigo_barras:
            preco_real = produto.preco / 100.0  # Converter de centavos para reais
            dados = [[produto.nome, produto.quantidade, f"R${preco_real:.2f}", produto.tipo.nome, produto.lote.numero, produto.validade.data, produto.codigo_barras]]
            messagebox.showinfo("Consulta de Item", tabulate(dados, headers=["Nome", "Quantidade", "Preço", "Tipo", "Lote", "Validade", "Código de Barras"], tablefmt="grid"))
            return
    messagebox.showerror("Erro", "Código de barras não encontrado.")

# Função para listar todos os itens do estoque
def listar_estoque():
    headers = ["Nome", "Quantidade", "Preço", "Tipo", "Lote", "Validade", "Código de Barras"]
    data = []
    for produto in estoque:
        preco_real = produto.preco / 100.0  # Converter de centavos para reais
        data.append([produto.nome, produto.quantidade, f"R${preco_real:.2f}", produto.tipo.nome, produto.lote.numero, produto.validade.data, produto.codigo_barras])

    # Configuração do quadro da tabela
    for row in tabela_estoque.get_children():
        tabela_estoque.delete(row)

    for row in data:
        tabela_estoque.insert("", "end", values=row)

# Função para verificar proximidade da validade
def verificar_validade():
    hoje = datetime.date.today()
    um_mes = datetime.timedelta(days=30)
    for produto in estoque:
        if hoje >= produto.validade.data:
            messagebox.showwarning("Aviso", f"O item '{produto.nome}' está vencido e será removido do estoque.")
            estoque.remove(produto)
    salvar_estoque_json()  # Salvar após verificar a validade

# Função para calcular o valor total dos volumes em estoque
def resumo_valor_total():
    valor_total = sum(produto.preco * produto.quantidade for produto in estoque) / 100.0  # Converter de centavos para reais
    mensagem = f"Valor total em estoque: R${valor_total:.2f}\n"

    tipos = {}
    for produto in estoque:
        if produto.tipo.nome not in tipos:
            tipos[produto.tipo.nome] = 0
        tipos[produto.tipo.nome] += produto.preco * produto.quantidade

    for tipo, valor em tipos.items():
        valor_real = valor / 100.0  # Converter de centavos para reais
        mensagem += f"Valor total para o tipo '{tipo}': R${valor_real:.2f}\n"

    messagebox.showinfo("Resumo do Valor Total", mensagem)

# Função para gerar gráfico informativo
def gerar_grafico():
    categorias = ["Saídas", "Perdas", "Entradas"]
    valores = [sum(produto.quantidade for produto in estoque), 0, sum(produto.quantidade for produto in estoque)]  # Placeholder para perdas
    plt.bar(categorias, valores)
    plt.xlabel('Categorias')
    plt.ylabel('Quantidade')
    plt.title('Gráfico Informativo')
    plt.show()

# Função para salvar o estoque em JSON
def salvar_estoque_json():
    dados_para_salvar = []
    for produto in estoque:
        dados_produto = {
            'nome': produto.nome,
            'quantidade': produto.quantidade,
            'preco': produto.preco,
            'tipo': produto.tipo.nome,
            'lote': produto.lote.numero,
            'validade': produto.validade.data.strftime("%d/%m/%Y"),
            'codigo_barras': produto.codigo_barras
        }
        dados_para_salvar.append(dados_produto)
    
    with open('estoque.json', 'w') as arquivo_json:
        json.dump(dados_para_salvar, arquivo_json, indent=4)

    messagebox.showinfo("Salvar Estoque", "Estoque salvo em 'estoque.json'.")

# Função para carregar o estoque de um arquivo JSON
def carregar_estoque_json():
    try:
        with open('estoque.json', 'r') as arquivo_json:
            conteudo = arquivo_json.read()
            dados = json.loads(conteudo)
            for item em dados:
                tipo = Tipo(item['tipo'])
                lote = Lote(item['lote'])
                validade = Validade(item['validade'])
                produto = Produto(item['nome'], item['quantidade'], item['preco'], tipo, lote, validade, item['codigo_barras'])
                estoque.append(produto)
        messagebox.showinfo("Carregar Estoque", "Estoque carregado de 'estoque.json'.")
    except FileNotFoundError:
        messagebox.showwarning("Aviso", "Arquivo 'estoque.json' não encontrado. Iniciando com estoque vazio.")
    except json.JSONDecodeError as e:
        messagebox.showerror("Erro", f"Erro ao carregar o JSON: {e}")

# Função para sair do programa
def sair_programa():
    salvar_estoque_json()
    root.quit()

# Função para criar a interface gráfica
def criar_interface():
    global root, tabela_estoque

    # Inicializar a janela principal
    root = ttk.Window(themename="superhero")
    root.title("Sistema de Gestão de Estoque")

    # Criar o frame para os botões
    frame = ttk.Frame(root)
    frame.pack(pady=20, padx=20)

    # Criar os botões e posicionar
    btn_cadastrar = ttk.Button(frame, text="Cadastrar Item", command=cadastrar_item, style='info.TButton')
    btn_cadastrar.grid(row=0, column=0, padx=10, pady=5)

    btn_remover = ttk.Button(frame, text="Remover Item", command=remover_item, style='info.TButton')
    btn_remover.grid(row=0, column=1, padx=10, pady=5)

    btn_adicionar = ttk.Button(frame, text="Adicionar Itens com Código de Barras", command=adicionar_item_codigo_barras, style='info.TButton')
    btn_adicionar.grid(row=0, column=2, padx=10, pady=5)

    btn_subtrair = ttk.Button(frame, text="Subtrair Itens com Código de Barras", command=subtrair_item_codigo_barras, style='info.TButton')
    btn_subtrair.grid(row=0, column=3, padx=10, pady=5)

    btn_consultar = ttk.Button(frame, text="Consultar Item pelo Código de Barras", command=consultar_item_codigo_barras, style='info.TButton')
    btn_consultar.grid(row=0, column=4, padx=10, pady=5)

    btn_listar = ttk.Button(frame, text="Listar Estoque", command=listar_estoque, style='info.TButton')
    btn_listar.grid(row=0, column=5, padx=10, pady=5)

    btn_resumo = ttk.Button(frame, text="Resumo do Valor Total", command=resumo_valor_total, style='info.TButton')
    btn_resumo.grid(row=0, column=6, padx=10, pady=5)

    btn_grafico = ttk.Button(frame, text="Gerar Gráfico Informativo", command=gerar_grafico, style='info.TButton')
    btn_grafico.grid(row=0, column=7, padx=10, pady=5)

    # Adicionando o botão "Salvar e Sair" na parte inferior direita
    btn_sair = ttk.Button(root, text="Salvar e Sair", command=sair_programa, style='info.TButton')
    btn_sair.pack(side="bottom", padx=20, pady=10, anchor='se')

    # Frame para a tabela
    frame_tabela = ttk.Frame(root)
    frame_tabela.pack(pady=10, padx=20, fill=BOTH, expand=True)  # Ajustando o padding para posicionar mais para cima

    # Configuração da tabela
    tabela_estoque = ttk.Treeview(frame_tabela, columns=["Nome", "Quantidade", "Preço", "Tipo", "Lote", "Validade", "Código de Barras"], show="headings", style="Custom.Treeview")
    
    # Configurar colunas para ajustar automaticamente ao conteúdo
    for col in tabela_estoque["columns"]:
        tabela_estoque.column(col, width=120, anchor=CENTER)  # Ajuste a largura conforme necessário

    for col in tabela_estoque["columns"]:
        tabela_estoque.heading(col, text=col)

    tabela_estoque.pack(fill=BOTH, expand=True)

    # Adicionar linhas divisórias singelas
    style = ttk.Style()
    style.configure("Custom.Treeview", rowheight=25, font=("Helvetica", 10), borderwidth=1, relief="solid", background="white")
    style.configure("Custom.Treeview.Heading", font=("Helvetica", 10, "bold"))
    style.layout("Custom.Treeview", [('Custom.Treeview.treearea', {'sticky': 'nswe'})])

    carregar_estoque_json()
    listar_estoque()
    verificar_validade()

    root.mainloop()

if __name__ == "__main__":
    criar_interface()
